# Amazon CloudWatch と AWS CloudTrail を使用した AgentCore Gateway のオブザーバビリティ設定

* Amazon CloudWatch は、AgentCore Gateway のリアルタイムパフォーマンス監視と運用トラブルシューティングに焦点を当て、レイテンシ、エラー率、使用パターンに関する詳細なメトリクスとログを提供します。
* AWS CloudTrail は、Gateway に関連する API 呼び出しとユーザーアクションの完全な履歴を記録することで、セキュリティ、コンプライアンス、監査に焦点を当てています。

これらを組み合わせることで、本番環境で AgentCore Gateway を管理するための包括的なオブザーバビリティとガバナンスフレームワークを提供します。

**Amazon CloudWatch**

主に AgentCore Gateway データプレーンのインタラクションをログに記録 - Gateway ツールのリスト取得（tools/list）、Gateway ツールの呼び出し（tools/call）、Gateway ツールの検索（tools/call）

| コンポーネントタイプ | 説明 |
| --- | --- |
| メトリクス | パフォーマンスと運用データ |
| トレース、スパン、リクエスト | リクエストの軌跡追跡 |
| アプリケーションログ | データプレーンの運用ログ |

**Amazon CloudTrail**

AgentCore Gateway コントロールプレーン（CreateGateway、ListGateway、DeleteGateway など）とデータプレーンインタラクション（InvokeGateway など）をログに記録

| コンポーネントタイプ | 説明 |
| --- | --- |
| 管理イベント | コントロールプレーンリクエストの ID 情報を含む、トレイル作成時にデフォルトで有効 |
| データイベント | リソースで実行された操作に関する情報。データイベントは通常大量のアクティビティ。明示的に有効にする必要あり。追加料金が発生。 |

## AgentCore Gateway の作成と AWS CloudTrail の監視

#### 前提条件

このチュートリアルを実行するには以下が必要です：
* Jupyter notebook（Python カーネル）
* uv
* AWS 認証情報
* Amazon Cognito

In [ ]:
!pip install --force-reinstall -U -r requirements.txt

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-west-2') # set the AWS region

In [ ]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

#### Lambda 関数の作成

**以下の出力から Lambda 関数 ARN をメモしてください**

In [ ]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools and note down the lambda ARN

lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")

if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("NOTE DOWN: Lambda function created with ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("NOTE DOWN: Lambda function creation failed with message: ", lambda_resp['lambda_function_arn'])

In [ ]:
#### Create an IAM role for the Gateway to assume
import utils
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

#### Gateway へのインバウンド認可用に Amazon Cognito Pool を作成

In [ ]:
# Creating Cognito User Pool 
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL  
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

#### インバウンド認可用に Amazon Cognito Authorizer を使用して Gateway を作成

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoGWforLambda',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with AWS Lambda target type'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

#### AWS Lambda ターゲットを作成し MCP ツールに変換

<font color="red"> **注意: 上記でメモした Lambda 関数 ARN に置き換えてください** </font>

In [ ]:
# Replace the AWS Lambda function ARN below
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": "<Lambda_ARN_noted_above>", # Replace this with your AWS Lambda function ARN noted above
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_tool",
                        "description": "tool to get the order",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    },                    
                    {
                        "name": "update_order_tool",
                        "description": "tool to update the orderId",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    }
                ]
            }
        }
    }
}

credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='LambdaUsingSDK'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target using SDK',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

## Amazon CloudWatch を使用した AgentCore Gateway オブザーバビリティの設定

### AgentCore Gateway アプリケーションログの設定

AgentCore Gateway アプリケーションログにエラーログが表示されるシナリオ：
- 実行ロールが bedrock-agentcore を信頼していない Gateway での MCP リクエスト tools/call
- 実行ロールがターゲットに関連付けられた CredentialProviderArn に対する適切な権限を持っていない Gateway での MCP リクエスト tools/call
- 実行ロールが Lambda 関数ターゲットを呼び出す適切な権限を持っていない Gateway での MCP リクエスト tools/call
- MCP リクエストに認可ヘッダーがない
- MCP リクエストに無効なベアラートークンがある（期限切れ、無効、クライアント ID が許可されていないなど）
- 存在しないツールへの MCP リクエスト tools/call

#### ステップ 0: ベンダーログ配信用の新しいロググループを作成

**CloudWatch ロググループ名をメモしてください**

In [ ]:
import boto3

# Initialize CloudWatch Logs client
logs_client = boto3.client('logs', region_name=REGION)
sts_client = boto3.client('sts', region_name=REGION)

cloudwatch_log_group = ""

# Define log group name
log_group_name = '/aws/vendedlogs/bedrock-agentcore/gateway/APPLICATION_LOGS/' + gatewayID
# Get AWS account ID
account_id = sts_client.get_caller_identity()['Account']
log_group_arn = f"arn:aws:logs:{REGION}:{account_id}:log-group:{log_group_name}:*"
print(f"Log Group ARN (constructed): {log_group_arn}")

try:
    # Create log group
    logs_client.create_log_group(logGroupName=log_group_name)
    print(f"NOTE DOWN:Successfully created log group: {log_group_name}")
    cloudwatch_log_group = log_group_name
    
except logs_client.exceptions.ResourceAlreadyExistsException:
    print(f"Log group {log_group_name} already exists")
except Exception as e:
    print(f"Error creating log group: {e}")


#### ステップ 1: ログの配信ソースを作成

In [ ]:
# Use PutDeliverySource to create a delivery source, which is a logical object that represents the resource that is actually sending the logs.
gateway_name=create_response["name"]
print(gateway_name)
delivery_source_response = logs_client.put_delivery_source(
            name = f"{gateway_name}-logs-source",
            logType='APPLICATION_LOGS',  # Specific to Bedrock
            resourceArn=create_response["gatewayArn"]
        )

#### ステップ 2: 配信先を作成

In [ ]:
# A delivery destination can represent a log group in CloudWatch Logs, an Amazon S3 bucket, a delivery stream in Firehose, or X-Ray. Here we are using CloudWatch log group.

delivery_destination_response = logs_client.put_delivery_destination(
            name='bedrock-agentcore-gw-destination',
            deliveryDestinationType='CWL',
            deliveryDestinationConfiguration = { 
              "destinationResourceArn": log_group_arn
           },
            outputFormat= "json",
        )
print(delivery_destination_response['deliveryDestination']['arn'])

#### ステップ 3: ログ配信を作成（ソースと配信先を接続）

In [ ]:
# Create a logs delivery by pairing the source and destination. A delivery is a connection between a logical delivery source and a logical delivery destination that you have already created.
delivery_response = logs_client.create_delivery(
            deliverySourceName=f"{gateway_name}-logs-source",
            deliveryDestinationArn=delivery_destination_response['deliveryDestination']['arn'],
            recordFields=['resource_arn','event_timestamp','body','account_id','timestamp','trace_id','span_id','request_id','gateway_id'],
)

In [ ]:
import time
time.sleep(10)

#### ステップ 4: AWS コンソールで確認

* AWS コンソールで [Amazon Bedrock AgentCore](https://console.aws.amazon.com/bedrock-agentcore/) サービスに移動します。
* AWS リージョンが正しいことを確認します。
* **Gateways** を選択します。
* 作成した Gateway を選択します。
* **Log deliveries and tracing** でエントリを確認します。

![Log deliveries](images/24-amazon-bedrock-agentcore-gw.png)

### ベンダーログ: AgentCore Gateway での tools/list 操作の呼び出し

#### インバウンド Gateway 認証用のトークン取得

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

#### Amazon CloudWatch でライブテーリングを有効化

In [ ]:
print(cloudwatch_log_group)

* **[Amazon CloudWatch コンソール](https://console.aws.amazon.com/cloudwatch/home)** に移動します
* **Log Groups** に移動します
* 上記の出力から特定のロググループを選択します（例: **/aws/vendedlogs/bedrock-agentcore/gateway/APPLICATION_LOGS/gatewayID**）
* **Start tailing** をクリックします

<img src="images/2-cloudwatch-live-tail.png" width="60%">

#### AgentCore Gateway での tools/list の呼び出し

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

with client:
    # Retrieve all tool capabilities from the gateway
    tools = client.list_tools_sync()
    for tool in tools:
        print(tool.tool_name)

#### バックエンドで何が起こっているか？

**上記の操作に対応して、3つのトレース ID、3つのリクエスト ID、3つのスパン ID を持つ8つのログメッセージがあります。**

**`trace_id`、`span_id`、`request_id` などの詳細を示すベンダーログからのフローシーケンス。** <br/>
**`gateway_id` も確認できます。**

**トレース ID:** トランザクション全体またはエージェントワークフローを表します — ユーザーの会話処理、オーケストレーション、ツール呼び出しの完全なフローをカバーします

**リクエスト ID:** トレースのコンテキスト内で AgentCore Gateway に対して行われた特定のリクエストを識別します。トランザクション中の異なる API 呼び出しやゲートウェイイベントを反映して、トレース内に複数のリクエスト ID が存在する場合があります。

**スパン ID:** 特定のリクエストに対して実行される特定のアクションまたは操作をマークします。各リクエストは複数のスパンを生成する可能性があり、それぞれが親リクエスト内のツール呼び出しやメモリイベントなどの詳細なステップを表す独自のスパン ID を持ちます

**MCP ハンドシェイク、ツールリストリクエストを示すシーケンス:**

3つのリクエストとスパン ID を持つ3つのトレース ID があります：

<font color="blue">トレース ID: 1 -> MCP ハンドシェイク <br/></font>
<font color="purple">トレース ID: 2 -> ハンドシェイク後の準備完了 - notifications/initialized <br/></font>
<font color="green">トレース ID: 3 -> Tools/list リクエスト / レスポンス<br/></font>

![overall_sequence](images/20-traceids-sequence.png)


**トレース ID: 68eb191773c47c5f6babe5cb4f81bd69**

* <font color="blue">クライアント → Gateway: 「こんにちは、私は MCP クライアント v0.1.0 です。プロトコルバージョン 2025-06-18 で通信できますか」</font> <br/><br/>
<img src="images/3-sequence1.png" width="60%">
* <font color="blue">Gateway → クライアント: 「はい、通信できます。」</font> <br/><br/>
<img src="images/4-sequence2.png" width="60%">
* <font color="blue"> Gateway → クライアント: 「これが私の情報です」 </font> <br/><br/>
<img src="images/5-sequence3.png" width="60%">

**トレース ID: 68eb1917268b63ef1290ed317254df8c**
* <font color="purple">クライアント → Gateway: 「初期化が成功した後、クライアントは準備完了を示す通知を送信します：」</font> <br/><br/>
<img src="images/6-sequence4.png" width="60%">
* <font color="purple">Gateway → クライアント: 「システムが準備完了したという通知を受信しました」</font> <br/><br/>
<img src="images/7-sequence5.png" width="60%">

**トレース ID: 68eb19177631f5357c06be8c182a99df**
* <font color="green">クライアント → Gateway: 「list/tools を呼び出しています」</font> <br/><br/>
<img src="images/8-sequence6.png" width="60%">
* <font color="green">Gateway → クライアント: 「ツールリストリクエストを受信しました」</font> <br/><br/>
<img src="images/9-sequence7.png" width="60%">
* <font color="green">Gateway → クライアント: 「こちらがツールです: get_order_tool と update_order_tool」</font> (完全なレスポンス)<br/><br/>
<img src="images/10-sequence8.png" width="60%">

### コンソールを使用して CloudWatch へのトレース配信を設定

このセクションでは、CloudWatch へのトレース配信を有効にして、アプリケーション全体のインタラクションの流れを追跡する方法を説明します。これにより、リクエストを可視化し、パフォーマンスのボトルネックを特定し、エラーをトラブルシュートし、パフォーマンスを最適化できます。

#### ステップ 1: トレースの配信ソースを作成

In [ ]:
traces_source_response = logs_client.put_delivery_source(
    name=f"{gateway_name}-traces-source", 
    logType="TRACES",
    resourceArn=create_response["gatewayArn"]
)

#### ステップ 2: 配信先を作成

In [ ]:
# A delivery destination can represent a log group in CloudWatch Logs, an Amazon S3 bucket, a delivery stream in Firehose, or X-Ray. Here we are using CloudWatch log group.

traces_destination_response = logs_client.put_delivery_destination(
            name=f"{gateway_name}-traces-destination",
            deliveryDestinationType='XRAY'
        )
print(traces_destination_response['deliveryDestination']['arn'])

#### ステップ 3: トレース配信を作成（ソースと配信先を接続）

In [ ]:
# Create a logs delivery by pairing the source and destination. A delivery is a connection between a logical delivery source and a logical delivery destination that you have already created.
delivery_response = logs_client.create_delivery(
            deliverySourceName=f"{gateway_name}-traces-source",
            deliveryDestinationArn=traces_destination_response['deliveryDestination']['arn']
)

In [ ]:
import time
time.sleep(10)

#### ステップ 4: AWS コンソールで確認

* AWS コンソールで [Amazon Bedrock AgentCore](https://console.aws.amazon.com/bedrock-agentcore/) サービスに移動します。
* AWS リージョンが正しいことを確認します。
* **Gateways** を選択します。
* 作成した Gateway を選択します。
* **Tracing** が `Enabled` になっていることを確認します。

![enable-tracing](images/26-enable-tracing.png)

### GenAI オブザーバビリティダッシュボードでのトレース - Gateway レベル

#### AgentCore Gateway での tools/list 操作の呼び出し

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL, headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

# Define specific tool parameters
TOOL_NAME = "LambdaUsingSDK___get_order_tool"
ORDER_ID = "123"

with client:
    try:
        # Call the get_order_tool
        result = client.call_tool_sync(
            tool_use_id="get-order-id-123-call-1",
            name=TOOL_NAME,
            arguments={"orderId": ORDER_ID}
        )
        
        # Print the tool's response
        print(f"\nGet Order Tool Response for Order ID {ORDER_ID}:")
        print(f"Content: {result['content'][0]['text']}")
                
    except Exception as e:
        print(f"Error occurred while calling {TOOL_NAME}: {str(e)}")

#### Amazon CloudWatch Gateway トレースとスパン

<font color="red"> **注意: Gateway とトレースが以下の GenAI オブザーバビリティダッシュボードに反映されるまで少なくとも2〜5分かかります。** </font>

In [ ]:
print(cloudwatch_log_group)

* **[Amazon CloudWatch コンソール](https://console.aws.amazon.com/cloudwatch/home)** に移動します
* **Log Groups** に移動します
* 上記の出力から特定のロググループを選択します（例: **/aws/vendedlogs/bedrock-agentcore/gateway/APPLICATION_LOGS/gatewayID**）
* `Search all log streams` を選択します
* CloudWatch ログで、`LambdaUsingSDK___get_order_tool` キーワードを検索します。<br/>
* 以下のスクリーンショットに示す操作に対応するトレース ID を特定してメモしてください。

例: **68ed6dc01c0556e2735177ed3794422a**

![genai-cw-logs](images/12-cloudwatch-logs-mcptool-2.png)

* **GenAI Observability** -> **Bedrock AgentCore** に移動します
* **Gateways** を選択します

![genai-obs-gw](images/11-cloudwatch-gateways.png)

* **Traces** を選択し、トレース ID: **68ed6dc01c0556e2735177ed3794422a** を検索します。
* **Trace ID** をクリックしてスパンとレイテンシを表示します。この例では、`InvokeTool` 操作が 580ms かかり、平均スパンレイテンシは 290ms であることがわかります。

![genai-obs-traces](images/13-cloudwatch-genai-traces-span.png)

**注意: トレースが表示されない場合は、右上隅の時間ウィンドウを適宜調整する必要があるかもしれません。**

![timer-window](images/25-bedrock-timewindow.png)

* トレースをさらに下にスクロールして、`span metadata` の詳細を確認します: <br/>

   `kind:SERVER` - 全体的な実行詳細、呼び出されたツール、Gateway の詳細、AWS リクエスト ID、トレースおよびスパン ID を追跡します。<br/>
   `kind:CLIENT` - 呼び出された特定のターゲットと、ターゲットタイプ、ターゲット実行時間、ターゲット実行開始・終了時刻などの詳細をカバーします。<br/>

  以下のスクリーンショットは、ツール実行に 378ms（`execute_tool_latency_ms`）かかり、ツール実行を除いた Gateway の時間が 152 ms（`overhead_latency`）であることを `AgentCore.Gateway.InvokeTool` の下に示しています。

![span-metadata](images/16-span-metadata.png)

### Amazon CloudWatch を使用した問題の根本原因の検出

**シナリオ:** Gateway に無効なトークンを送信し、ログとスパンを確認します。

#### 無効なトークンの設定

In [ ]:
token1 = "12345"

In [ ]:
print(cloudwatch_log_group)

#### CloudWatch ログのライブテーリングを開始

* **[Amazon CloudWatch コンソール](https://console.aws.amazon.com/cloudwatch/home)** に移動します
* **Log Groups** に移動します
* 上記の出力から特定のロググループを選択します（例: **/aws/vendedlogs/bedrock-agentcore/gateway/APPLICATION_LOGS/gatewayID**）
* **Start tailing** をクリックします

![cloudwatch_tail](images/2-cloudwatch-live-tail.png)

#### 無効なトークンで特定の Gateway ツールを呼び出す

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL, headers={"Authorization": f"Bearer {token1}"})

client = MCPClient(create_streamable_http_transport)

# Define specific tool parameters
TOOL_NAME = "LambdaUsingSDK___get_order_tool"
ORDER_ID = "123"

with client:
    try:
        # Call the get_order_tool
        result = client.call_tool_sync(
            tool_use_id="get-order-id-123-call-1",
            name=TOOL_NAME,
            arguments={"orderId": ORDER_ID}
        )
        
        # Print the tool's response
        print(f"\nGet Order Tool Response for Order ID {ORDER_ID}:")
        print(f"Content: {result['content'][0]['text']}")
                
    except Exception as e:
        print(f"Error occurred while calling {TOOL_NAME}: {str(e)}")

#### Amazon CloudWatch ログでの詳細情報

生成された例外は**問題の根本原因に関する有用な情報を提供しません**：

    raise MCPClientInitializationError("the client initialization failed") from e
    strands.types.exceptions.MCPClientInitializationError: the client initialization failed


例外の背後にある根本原因を理解するには、CloudWatch ログを確認してください。

![rootcause](images/17-rootcause.png)

#### トレースとスパンでの詳細情報

* 上記のログから `trace_id` の値をメモしてください。
* CloudWatch コンソールで、**GenAI Observability** -> **Bedrock AgentCore** に移動します
* **Gateways** を選択します
* **Traces** を選択し、トレース ID: **68ee6edf04de45005c702392328eb065** を検索します。
* 以下のスクリーンショットに示すように、スパンで詳細を確認できます。

![troubleshooting-traces](images/18-troubleshooting-traces1.png)

### AgentCore Gateway CloudWatch メトリクス

* **[Amazon CloudWatch コンソール](https://console.aws.amazon.com/cloudwatch/home)** に移動します
* **Metrics** -> **All metrics** を選択します
* **AWS namespaces** -> **Bedrock-AgentCore** を選択します
* **Method, Name, Operation, Protocol, Resource** を選択します。操作レベルで集計するには、**Method, Operation, Protocol, Resource** を選択できます。
* ダッシュボードを作成するか、値を表示するために任意のメトリクスを選択します。

![19-metrics](images/19-metrics.png)

### エージェントトレースの理解 - AgentCore Runtime 上の Strands Agent が AgentCore Gateway に接続

#### 以下の必須変数の値をダブルクォート付きでメモしてください

In [ ]:
print('mcp_url: "%s"' % gatewayURL)
print('user_pool_id: "%s"' % user_pool_id)
print('client_id: "%s"' % client_id)
print('client_secret: "%s"' % client_secret)
print('scopeString: "%s"' % scopeString)
token_endpoint = f"https://{user_pool_id.replace('_', '')}.auth.{REGION}.amazoncognito.com/oauth2/token"
print('token_endpoint: "%s"' % token_endpoint)
print('region: "%s"' % REGION)

#### AgentCore Runtime 経由でデプロイする Bedrock Agent のコード

<font color="red"> コード内の **mcp_url**、**user_pool_id**、**client_id**、**client_secret**、**scopeString**、**token_endpoint**、**region** の値を置き換えてください。</font>

以下の Python コードは、AWS Bedrock AgentCore Gateway に接続して Model Context Protocol (MCP) を通じて外部ツールにアクセスし使用する Strands エージェントを作成します。エージェントは AWS Cognito 資格情報を使用して認証し、Gateway から利用可能なツールを取得し、Claude 言語モデルを通じてそれらのツールを呼び出してユーザーのクエリを処理します。

In [ ]:
%%writefile strands_agent_gateway.py
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from datetime import datetime
from strands import Agent, tool
import logging
from strands.models import BedrockModel
from strands.tools import mcp
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
import os
import time
import functools
import asyncio
import json
import argparse

#### Substitute the values below with ones from above output:

# GateWay URL:
mcp_url = #<mcp_url> # Replace this. For example: "https://1demogatewayforlambda-tpwablbixre.gateway.bedrock-agentcore.us-west-2.amazonaws.com/mcp"
# # Cognito parameters:
user_pool_id =  #<user_pool_id> # Replace this. For example: "us-west-2_CpyXraYjW"
client_id = #<client_id> # Replace this. For example: "5ifm4heh1r3oa19r2mnngsvung"
client_secret =  #<client_secret> # Replace this. For example: "kchfnio0inegso43jrc2js0ntgqc5ku2uplhd9v7vp0bo8sp1g0"
scopeString =  #<scopeString> # Replace this. For example: "sample-agentcore-gateway-id/gateway:read sample-agentcore-gateway-id/gateway:write"
token_endpoint =  #<token_endpoint> # Replace this. For example: "https://us-west-2_CpyXraYjW.auth.us-west-2.amazoncognito.com/oauth2/token"
region =  #<region> # Replace this. For example: "us-west-2"

###########################################

model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
bedrockmodel = BedrockModel(
    inference_profile_id= model_id,
    temperature=0,
    streaming=True,
)

# Defining  client as our GatewayClient
client = GatewayClient(region_name=region)
client.logger.setLevel(logging.DEBUG)

# Get token
client_config = {
    "user_pool_id": user_pool_id,
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": scopeString,
    "region": region,
    "token_endpoint": token_endpoint
}

token_response = client.get_access_token_for_cognito(client_config)
access_token = token_response
print(access_token)


# Get Gateway tools
def create_streamable_http_transport(mcp_url: str, access_token: str):
    return streamablehttp_client(mcp_url, headers={"Authorization": f"Bearer {access_token}"})
  
def get_full_tools_list(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True 
            pagination_token = tmp_tools.pagination_token
    return tools

def run_agent(mcp_url: str, access_token: str, user_message: str):
    try:
        mcp_client = MCPClient(lambda: create_streamable_http_transport(mcp_url, access_token))
        #region="us-east-1"
        
        with mcp_client:
            tools = get_full_tools_list(mcp_client)
            print(f"Found the following tools: {[tool.tool_name for tool in tools]}")
            agent = Agent(model=bedrockmodel,tools=tools, callback_handler=None)
            print("\nThinking...\n")
            print(user_message)
            result = agent(user_message)        
        return result            
    except Exception as e:
        print(f"Error in run_agent: {e}")
        return {"message": f"Error processing request: {str(e)}"}
        
        
# Define our app referencing the pre-built BedrockAgentCoreApp
app = BedrockAgentCoreApp()

@app.entrypoint
def invoke(payload):
    try:
        """Process user input and return a response"""
        user_message = payload.get("prompt", "Hello")

        result = run_agent(mcp_url, access_token, user_message)
        print(result)
        return {"result": result.message}
    except Exception as e:
        return {"result": f"Error: {str(e)}"}

if __name__ == "__main__":
    app.run()

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_demo_agent"
response = agentcore_runtime.configure(
    entrypoint="strands_agent_gateway.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

In [ ]:
launch_result = agentcore_runtime.launch()

In [ ]:
import time
time.sleep(20)

#### 「Memory is still provisioning (current status: CREATING). Short-term memory takes 30-90 seconds to activate.」というエラーが表示された場合は、リトライする前にさらに数秒待ってください。

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "list all tools"})
invoke_response

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "Check the order information for order id 123"})
invoke_response

#### Amazon Cloudwatch でトレースを確認 - エージェントレベル

* **[Amazon CloudWatch コンソール](https://console.aws.amazon.com/cloudwatch/home)** に移動します
* **GenAI Observability** -> **Bedrock AgentCore** に移動します
* **Agents** -> **Traces** を選択します
* **2つのトレース**があります。各プロンプト（list/tools と特定のツール呼び出し）に1つずつです。

**注意: トレースが表示されない場合は、右上隅の時間ウィンドウを適宜調整する必要があるかもしれません。**

![timer-window](images/25-bedrock-timewindow.png)

![Traces](images/14-traces-agentlevel-1.png)
![Traces](images/27-invoketools-agent-span.png)

#### トレースのスパンを確認

![Traces](images/15-spans-agent-1.png)

## AWS CloudTrail を使用したオブザーバビリティ

#### CloudTrail ログを保存するための一意の S3 バケット名の生成

In [ ]:
import boto3
import uuid

def generate_s3_bucket_name(base_name):
    account_id = boto3.client('sts').get_caller_identity()['Account']
    unique_id = uuid.uuid4().hex[:8]
    return f"{base_name}-{account_id}-{unique_id}".lower()

#### S3 バケットを作成する関数

In [ ]:
import boto3
from botocore.exceptions import ClientError

def create_bucket(bucket_name, region):
    try:
        s3_client = boto3.client('s3', region_name=region)
        if region == 'us-east-1':
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
        print(f"Bucket '{bucket_name}' created in region '{region}'")
        return True
    except ClientError as e:
        print(f"Error creating bucket: {e}")
        return False


#### CloudTrail がログを保存できるようにする S3 バケットポリシーを作成する関数

In [ ]:
import boto3
import json

def put_bucket_policy(bucket_name, account_id, region, trail_name):
    s3_client = boto3.client('s3')
    trail_arn = f"arn:aws:cloudtrail:{region}:{account_id}:trail/{trail_name}"
    bucket_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "AWSCloudTrailAclCheck20150319",
                "Effect": "Allow",
                "Principal": {"Service": "cloudtrail.amazonaws.com"},
                "Action": "s3:GetBucketAcl",
                "Resource": f"arn:aws:s3:::{bucket_name}",
                "Condition": {
                    "StringEquals": {"aws:SourceArn": trail_arn}
                }
            },
            {
                "Sid": "AWSCloudTrailWrite20150319",
                "Effect": "Allow",
                "Principal": {"Service": "cloudtrail.amazonaws.com"},
                "Action": "s3:PutObject",
                "Resource": f"arn:aws:s3:::{bucket_name}/AWSLogs/{account_id}/*",
                "Condition": {
                    "StringEquals": {
                        "s3:x-amz-acl": "bucket-owner-full-control",
                        "aws:SourceArn": trail_arn
                    }
                }
            }
        ]
    }
    policy_string = json.dumps(bucket_policy)
    s3_client.put_bucket_policy(Bucket=bucket_name, Policy=policy_string)
    print(f"Bucket policy set for bucket '{bucket_name}' with trail '{trail_name}'.")
    

#### CloudTrail ログ用の CloudWatch ロググループを作成する関数

In [ ]:
def create_cloudwatch_log_group(log_group_name, account_id, region):
    """Create CloudWatch Logs group and CloudTrail log stream"""
    logs_client = boto3.client('logs')
    log_stream_name = f"{account_id}_CloudTrail_{region}"
    
    try:
        # Create log group
        logs_client.create_log_group(logGroupName=log_group_name)
        print(f"Created CloudWatch Logs group: {log_group_name}")
        
        # Create log stream
        logs_client.create_log_stream(
            logGroupName=log_group_name,
            logStreamName=log_stream_name
        )
        print(f"Created CloudWatch Logs stream: {log_stream_name}")
        
    except logs_client.exceptions.ResourceAlreadyExistsException as e:
        if "Log Group" in str(e):
            print(f"CloudWatch Logs group already exists: {log_group_name}")
            # Try to create log stream even if group exists
            try:
                logs_client.create_log_stream(
                    logGroupName=log_group_name,
                    logStreamName=log_stream_name
                )
                print(f"Created CloudWatch Logs stream: {log_stream_name}")
            except logs_client.exceptions.ResourceAlreadyExistsException:
                print(f"CloudWatch Logs stream already exists: {log_stream_name}")
        else:
            print(f"CloudWatch Logs stream already exists: {log_stream_name}")

# Usage in main code
account_id = boto3.client('sts').get_caller_identity()['Account']
trail_name = 'AgentCoreGatewayMgmtTrail'
log_group_name = f"/aws/cloudtrail/{trail_name}"


#### CloudTrail 用の IAM ロールを作成する関数

In [ ]:
def create_cloudwatch_role(role_name, account_id, region, log_group_name):
    """Create IAM role for CloudTrail to CloudWatch Logs"""
    iam = boto3.client('iam')
    
    # Trust policy for CloudTrail
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"Service": "cloudtrail.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }]
    }
    
    # CloudWatch Logs policy with specific log group name
    role_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "AWSCloudTrailCreateLogStream2014110",
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogStream"
                ],
                "Resource": [
                    f"arn:aws:logs:{region}:{account_id}:log-group:{log_group_name}:log-stream:{account_id}_CloudTrail_{region}*"
                ]
            },
            {
                "Sid": "AWSCloudTrailPutLogEvents20141101",
                "Effect": "Allow",
                "Action": [
                    "logs:PutLogEvents"
                ],
                "Resource": [
                    f"arn:aws:logs:{region}:{account_id}:log-group:{log_group_name}:log-stream:{account_id}_CloudTrail_{region}*"
                ]
            }
        ]
    }
    
    try:
        # Create the role
        role = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy)
        )
        
        # Attach the policy
        iam.put_role_policy(
            RoleName=role_name,
            PolicyName=f"{role_name}-policy",
            PolicyDocument=json.dumps(role_policy)
        )
        
        # Wait briefly for role to be available
        import time
        time.sleep(5)
        
        return role['Role']['Arn']
    except iam.exceptions.EntityAlreadyExistsException:
        return iam.get_role(RoleName=role_name)['Role']['Arn']


In [ ]:
account_id = boto3.client('sts').get_caller_identity()['Account']
trail_name = 'AgentCoreGatewayMgmtTrail'
role_name = f"CloudTrail-{trail_name}-{REGION}"
print(role_name)
log_group_name = f"/aws/cloudtrail/{trail_name}"
print(log_group_name)

#Creating CloudWatchRole
cloudwatch_role_arn = create_cloudwatch_role(role_name, account_id, REGION, log_group_name)
print(cloudwatch_role_arn)

#### IAM ロールが伝播するまで待機

In [ ]:
import time
time.sleep(20)

#### S3 バケットと CloudWatch ロググループの作成

In [ ]:
import boto3

# Create S3 bucket for CloudTrail logs
s3_bucket_name = generate_s3_bucket_name("agentcore-gateway")
print(s3_bucket_name)
create_bucket(s3_bucket_name, REGION)
put_bucket_policy(s3_bucket_name, account_id, REGION, trail_name)

# Create CloudWatch Logs group
create_cloudwatch_log_group(log_group_name, account_id, REGION)

In [ ]:
import time
time.sleep(20)

#### AgentCore Gateway の管理イベントをログ記録する CloudTrail の作成

In [ ]:
# Creating cloudtrail
cloudtrail_client = boto3.client('cloudtrail', region_name=REGION)

# Create the trail
response = cloudtrail_client.create_trail(
    Name=trail_name,
    S3BucketName=s3_bucket_name,
    CloudWatchLogsLogGroupArn=f"arn:aws:logs:{REGION}:{account_id}:log-group:{log_group_name}:*",
    CloudWatchLogsRoleArn=cloudwatch_role_arn
)

# Define advanced event selector to only include management events for AgentCore Gateway
advanced_event_selectors = [
    {
        'Name': 'AgentCoreGatewayManagementEvents',
        'FieldSelectors': [
            {
                'Field': 'eventCategory',
                'Equals': ['Management']
            }
        ]
    }
]

# Update the trail to use the advanced event selector
cloudtrail_client.put_event_selectors(
    TrailName=trail_name,
    AdvancedEventSelectors=advanced_event_selectors
)

# Start logging events
cloudtrail_client.start_logging(Name=trail_name)

print(f"CloudTrail trail '{trail_name}' created and logging management events for AgentCore Gateway.")

#### AWS コンソールで確認

* AWS コンソールで [AWS CloudTrail](https://console.aws.amazon.com/cloudtrailv2/) サービスに移動します
* AWS リージョンが正しいことを確認します。
* Trails をクリックして `AgentCoreGatewayMgmtTrail` が正常に作成されていることを確認します。

![ManagementTrail](images/28-cloudtrail.png)

In [ ]:
import time
time.sleep(20)

#### tools/list を呼び出して CloudTrail イベントで API 追跡を確認

In [ ]:
def list_gateways():
    """List all Bedrock Agent Core gateways"""
    try:
        # Initialize Bedrock Agent Core client
        gateway_client = boto3.client('bedrock-agentcore-control', 
                                    region_name=REGION)

        print(REGION)
        
        # List gateways
        response = gateway_client.list_gateways()
        print(response)
        
        # Print gateway details
        if 'items' in response and response['items']:
            print("\nGateways found:")
            for gateway in response['items']:
                print(f"\nName: {gateway.get('name')}")
                print(f"Gateway ID: {gateway.get('gatewayId')}")
                print(f"Gateway URL: {gateway.get('gatewayUrl')}")
                print(f"Status: {gateway.get('status')}")
                print(f"Protocol Type: {gateway.get('protocolType')}")
                print(f"Authorizer Type: {gateway.get('authorizerType')}")
        else:
            print("No gateways found")
            
        return response.get('gateways', [])
            
    except Exception as e:
        print(f"Error listing gateways: {str(e)}")
        return []

# Call the function
gateways = list_gateways()

In [ ]:
import time
time.sleep(20)

<font color="red"> 注意: ログエントリが CloudTrail ログに反映されるまで数秒かかる場合があります</font>

#### CloudTrail イベントを確認

* **[Amazon CloudWatch コンソール](https://console.aws.amazon.com/cloudwatch/home)** に移動します <br/>
* CloudTrail Log Groups -> `/aws/cloudtrail/AgentCoreGatewayMgmtTrail` -> `Search all log streams` を選択します <br/>
* 検索ボックスで `ListGateways` を検索します。

**タイプ `IAMUser` の ListGateways API 呼び出し** <br/><br/>
![ListGateways](images/22-list-gateways-1.png)

同様に、CreateGateway 操作やその他の [Gateway イベント](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway-event-types.html) も管理イベントの下にログ記録されます。 <br/> <br/>
![ManagementEvents](images/21-cloudtrail-mgmt.png)

#### CloudTrail でデータイベントをログ記録

**<font color="red"> 注意: データイベントは追加コストが発生し、ボリュームが多いためデフォルトでは有効になっていません</font>**

In [ ]:
account_id = boto3.client('sts').get_caller_identity()['Account']
trail_name = 'AgentCoreGatewayDataTrail'
role_name = f"CloudTrail-{trail_name}-{REGION}"
print(role_name)
log_group_name = f"/aws/cloudtrail/{trail_name}"
print(log_group_name)

#Creating CloudWatchRole
cloudwatch_role_arn = create_cloudwatch_role(role_name, account_id, REGION, log_group_name)
print(cloudwatch_role_arn)

In [ ]:
import time
time.sleep(20)

In [ ]:
import boto3

# Create S3 bucket for CloudTrail logs
s3_bucket_name = generate_s3_bucket_name("agentcore-gateway-data")
print(s3_bucket_name)
create_bucket(s3_bucket_name, REGION)
put_bucket_policy(s3_bucket_name, account_id, REGION, trail_name)

# Create CloudWatch Logs group
create_cloudwatch_log_group(log_group_name, account_id, REGION)

In [ ]:
import time
time.sleep(20)

In [ ]:
# Creating cloudtrail
cloudtrail_client = boto3.client('cloudtrail', region_name=REGION)

# Create the trail
response = cloudtrail_client.create_trail(
    Name=trail_name,
    S3BucketName=s3_bucket_name,
    CloudWatchLogsLogGroupArn=f"arn:aws:logs:{REGION}:{account_id}:log-group:{log_group_name}:*",
    CloudWatchLogsRoleArn=cloudwatch_role_arn
)

# Define advanced event selector to only include management events for AgentCore Gateway
advanced_event_selectors = [
    {
        'Name': 'AgentCoreGatewayDataEvents',
        'FieldSelectors': [
            {
                'Field': 'eventCategory',
                'Equals': ['Data']
            },
            {
                'Field': 'resources.type',
                'Equals': ['AWS::BedrockAgentCore::Gateway']  # Corrected service name
            }
        ]
    }
]

# Update the trail to use the advanced event selector
cloudtrail_client.put_event_selectors(
    TrailName=trail_name,
    AdvancedEventSelectors=advanced_event_selectors
)

# Start logging events
cloudtrail_client.start_logging(Name=trail_name)

print(f"CloudTrail trail '{trail_name}' created and logging data events for AgentCore Gateway.")

#### AWS コンソールで確認

* AWS コンソールで [AWS CloudTrail](https://console.aws.amazon.com/cloudtrailv2/) サービスに移動します
* AWS リージョンが正しいことを確認します。
* Trails をクリックして `AgentCoreGatewayDataTrail` が正常に作成されていることを確認します。


![ManagementTrail](images/29-data-cloudtrail1.png)

In [ ]:
import time
time.sleep(20)

#### AgentCore Gateway 用のトークンを取得

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient

print(gatewayURL)

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL, headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

# Define specific tool parameters
TOOL_NAME = "LambdaUsingSDK___get_order_tool"
ORDER_ID = "123"

with client:
    try:
        # Call the get_order_tool
        result = client.call_tool_sync(
            tool_use_id="get-order-id-123-call-1",
            name=TOOL_NAME,
            arguments={"orderId": ORDER_ID}
        )
        
        # Print the tool's response
        print(f"\nGet Order Tool Response for Order ID {ORDER_ID}:")
        print(f"Content: {result['content'][0]['text']}")
                
    except Exception as e:
        print(f"Error occurred while calling {TOOL_NAME}: {str(e)}")

#### CloudTrail イベントを確認

* **[Amazon CloudWatch コンソール](https://console.aws.amazon.com/cloudwatch/home)** に移動します <br/>
* CloudTrail Log Groups -> `/aws/cloudtrail/AgentCoreGatewayDataTrail` -> `Search all log streams` を選択します <br/>

<font color="red"> 注意: ログエントリが CloudTrail ログに反映されるまで数秒かかる場合があります</font>

![cloudtrail-data](images/23-cloudtrail-data.png)

# クリーンアップ

このスクリプトは作成されたほぼすべてのリソースを削除します。ただし、一部のリソースが削除されない場合や削除に失敗した場合は、手動で削除する必要があるかもしれません。

In [ ]:
import boto3
import os
import time
import json
from botocore.exceptions import ClientError

# Initialize AWS clients
REGION = os.environ.get('AWS_DEFAULT_REGION', 'us-east-1')
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
logs_client = boto3.client('logs', region_name=REGION)
cloudtrail_client = boto3.client('cloudtrail', region_name=REGION)
s3_client = boto3.client('s3', region_name=REGION)
iam_client = boto3.client('iam', region_name=REGION)
cognito_client = boto3.client('cognito-idp', region_name=REGION)
lambda_client = boto3.client('lambda', region_name=REGION)
ecr_client = boto3.client('ecr', region_name=REGION)
sts_client = boto3.client('sts', region_name=REGION)

account_id = sts_client.get_caller_identity()['Account']

def get_agentcore_runtime_id_by_name(agent_name, region):
    """
    Get AgentCore Runtime ID by agent name.
    
    :param agent_name: The name of the agent runtime to find
    :param region: AWS region where the runtime exists
    :return: The runtime ID if found, None otherwise
    """
    agent_runtime_id = None
    
    try:
        client = boto3.client('bedrock-agentcore-control', region_name=region)
        response = client.list_agent_runtimes()
        print(f"Searching for agent runtime matching: {agent_name}")
        
        for runtime in response.get('agentRuntimes', []):
            if runtime.get('agentRuntimeId', '').find(agent_name) >= 0:
                agent_runtime_id = runtime.get('agentRuntimeId')
                print(f"Found runtime ID: {agent_runtime_id}")
                print(f"Found runtime: {agent_name}")
                break
        
        return agent_runtime_id
        
    except ClientError as e:
        print(f"Error listing runtimes: {e}")
        raise

def delete_agentcore_runtime(region, agent_runtime_id):
    """Delete AgentCore runtime using the correct runtime ID"""
    try:
        client = boto3.client('bedrock-agentcore-control', region_name=region)
        response = client.delete_agent_runtime(
            agentRuntimeId=agent_runtime_id
        )
        print(f"Successfully deleted AgentCore Runtime: {agent_runtime_id}")
        return response
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"AgentCore runtime {agent_runtime_id} not found - may already be deleted")
        elif error_code == 'ConflictException':
            print(f"Cannot delete AgentCore runtime {agent_runtime_id} - resource is in use")
        elif error_code == 'AccessDeniedException':
            print(f"Access denied. Ensure caller has bedrock-agentcore:DeleteAgentRuntime permission")
        else:
            print(f"Error deleting AgentCore runtime: {e}")
        raise

def delete_ecr_repository(ecr_repository_name):
    """Delete ECR repository if name is provided"""
    try:
        print(f"Deleting ECR repository: {ecr_repository_name}")
        ecr_response = ecr_client.delete_repository(
            repositoryName=ecr_repository_name,
            force=True  # Force delete even if images exist
        )
        print(f"ECR repository {ecr_repository_name} deleted successfully")
        return ecr_response
    except ClientError as e:
        print(f"Error deleting ECR repository {ecr_repository_name}: {e}")
        raise

def delete_gateway_and_targets(gateway_id):
    """Delete gateway targets and the gateway itself"""
    try:
        # List and delete all gateway targets
        targets_response = gateway_client.list_gateway_targets(gatewayIdentifier=gateway_id)
        for target in targets_response.get('items', []):
            target_id = target['targetId']
            print(f"Deleting gateway target: {target_id}")
            gateway_client.delete_gateway_target(
                gatewayIdentifier=gateway_id,
                targetIdentifier=target_id
            )
            time.sleep(2)
        
        # Delete the gateway
        print(f"Deleting gateway: {gateway_id}")
        gateway_client.delete_gateway(gatewayIdentifier=gateway_id)
        print(f"Gateway {gateway_id} deleted successfully")
        
    except ClientError as e:
        print(f"Error deleting gateway: {e}")

def delete_cloudwatch_log_deliveries(gateway_name):
    """Delete CloudWatch log and trace deliveries"""
    try:
        logs_source_name = f"{gateway_name}-logs-source"
        traces_source_name = f"{gateway_name}-traces-source"
        
        # List and delete deliveries
        deliveries = logs_client.describe_deliveries()
        for delivery in deliveries.get('deliveries', []):
            delivery_id = delivery['id']
            print(f"Deleting delivery: {delivery_id}")
            logs_client.delete_delivery(id=delivery_id)
            time.sleep(1)
        
        # Delete delivery sources
        for source_name in [logs_source_name, traces_source_name]:
            try:
                print(f"Deleting delivery source: {source_name}")
                logs_client.delete_delivery_source(name=source_name)
            except ClientError as e:
                print(f"Error deleting source {source_name}: {e}")
        
        # Delete delivery destinations
        destinations = logs_client.describe_delivery_destinations()
        for dest in destinations.get('deliveryDestinations', []):
            dest_name = dest['name']
            if 'bedrock-agentcore-gw' in dest_name or gateway_name in dest_name:
                print(f"Deleting delivery destination: {dest_name}")
                logs_client.delete_delivery_destination(name=dest_name)
                time.sleep(1)
                
    except ClientError as e:
        print(f"Error deleting log deliveries: {e}")

def delete_cloudwatch_log_groups(gateway_id):
    """Delete CloudWatch log groups"""
    log_groups = [
        f'/aws/vendedlogs/bedrock-agentcore/gateway/APPLICATION_LOGS/{gateway_id}',
        '/aws/cloudtrail/AgentCoreGatewayMgmtTrail',
        '/aws/cloudtrail/AgentCoreGatewayDataTrail'
    ]
    
    for log_group in log_groups:
        try:
            print(f"Deleting log group: {log_group}")
            logs_client.delete_log_group(logGroupName=log_group)
            print(f"Log group {log_group} deleted")
        except ClientError as e:
            print(f"Error deleting log group {log_group}: {e}")

def delete_cloudtrail_trails():
    """Delete CloudTrail trails"""
    trails = ['AgentCoreGatewayMgmtTrail', 'AgentCoreGatewayDataTrail']
    
    for trail_name in trails:
        try:
            # Stop logging
            print(f"Stopping logging for trail: {trail_name}")
            cloudtrail_client.stop_logging(Name=trail_name)
            time.sleep(2)
            
            # Delete trail
            print(f"Deleting trail: {trail_name}")
            cloudtrail_client.delete_trail(Name=trail_name)
            print(f"Trail {trail_name} deleted")
            
        except ClientError as e:
            print(f"Error deleting trail {trail_name}: {e}")

def empty_and_delete_s3_bucket(bucket_name):
    """Empty and delete S3 bucket"""
    try:
        # List and delete all objects
        print(f"Emptying S3 bucket: {bucket_name}")
        paginator = s3_client.get_paginator('list_objects_v2')
        
        for page in paginator.paginate(Bucket=bucket_name):
            if 'Contents' in page:
                objects = [{'Key': obj['Key']} for obj in page['Contents']]
                s3_client.delete_objects(Bucket=bucket_name, Delete={'Objects': objects})
        
        # Delete the bucket
        print(f"Deleting S3 bucket: {bucket_name}")
        s3_client.delete_bucket(Bucket=bucket_name)
        print(f"S3 bucket {bucket_name} deleted")
        
    except ClientError as e:
        print(f"Error deleting S3 bucket {bucket_name}: {e}")

def delete_iam_roles():
    """Delete IAM roles created for CloudTrail and Gateway"""
    roles = [
        f"CloudTrail-AgentCoreGatewayMgmtTrail-{REGION}",
        f"CloudTrail-AgentCoreGatewayDataTrail-{REGION}",
        "sample-lambdagateway-role"
    ]
    
    for role_name in roles:
        try:
            # Delete inline policies
            policies = iam_client.list_role_policies(RoleName=role_name)
            for policy_name in policies.get('PolicyNames', []):
                print(f"Deleting inline policy {policy_name} from role {role_name}")
                iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
            
            # Detach managed policies
            attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)
            for policy in attached_policies.get('AttachedPolicies', []):
                print(f"Detaching policy {policy['PolicyArn']} from role {role_name}")
                iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy['PolicyArn'])
            
            # Delete the role
            print(f"Deleting IAM role: {role_name}")
            iam_client.delete_role(RoleName=role_name)
            print(f"IAM role {role_name} deleted")
            
        except ClientError as e:
            print(f"Error deleting IAM role {role_name}: {e}")

def delete_cognito_resources(user_pool_id, client_id):
    """Delete Cognito user pool and client"""
    try:
        # Delete app client
        print(f"Deleting Cognito app client: {client_id}")
        cognito_client.delete_user_pool_client(
            UserPoolId=user_pool_id,
            ClientId=client_id
        )
        
        # Delete resource server
        resource_server_id = "sample-agentcore-gateway-id"
        print(f"Deleting Cognito resource server: {resource_server_id}")
        cognito_client.delete_resource_server(
            UserPoolId=user_pool_id,
            Identifier=resource_server_id
        )
        
        # Delete user pool
        print(f"Deleting Cognito user pool: {user_pool_id}")
        cognito_client.delete_user_pool(UserPoolId=user_pool_id)
        print(f"Cognito user pool {user_pool_id} deleted")
        
    except ClientError as e:
        print(f"Error deleting Cognito resources: {e}")

def delete_lambda_function(function_name="gateway_lambda"):
    """Delete Lambda function"""
    try:
        print(f"Deleting Lambda function: {function_name}")
        lambda_client.delete_function(FunctionName=function_name)
        print(f"Lambda function {function_name} deleted")
    except ClientError as e:
        print(f"Error deleting Lambda function: {e}")

# Main cleanup execution
if __name__ == "__main__":
    print("=" * 80)
    print("AMAZON BEDROCK AGENTCORE GATEWAY OBSERVABILITY CLEANUP")
    print("=" * 80)
    
    # IMPORTANT: Replace these with your actual resource identifiers
    gateway_id = gatewayID  # Replace with actual gateway ID
    gateway_name = gatewayID  # Replace with actual gateway name
    user_pool_id = user_pool_id  # Replace with actual user pool ID
    client_id = client_id  # Replace with actual client ID
    agent_name = "strands_demo_agent"  # Agent name to search for
    ecr_repository_name = "bedrock-agentcore-" + agent_name # Replace with actual ECR repository name (optional)
    
    # Step 1: Lookup and Delete AgentCore Runtime Agent and ECR Repository
    print("\n[1/9] Looking up and Deleting AgentCore Runtime Agent and ECR Repository...")
    try:
        # Use your function to lookup the runtime ID by name
        agent_runtime_id = get_agentcore_runtime_id_by_name(agent_name, REGION)
        
        if agent_runtime_id:
            print(f"Region: {REGION}")
            # Delete the runtime using your delete function
            delete_agentcore_runtime(REGION, agent_runtime_id)
            time.sleep(5)
            
            # Delete ECR repository if provided
            if ecr_repository_name:
                delete_ecr_repository(ecr_repository_name)
            else:
                print("No ECR repository name provided - skipping ECR cleanup")
        else:
            print(f"No runtime found matching: {agent_name}")
            print("Skipping AgentCore runtime deletion...")
    except Exception as e:
        print(f"Failed to delete AgentCore runtime: {e}")
        print("Continuing with remaining cleanup tasks...")
    
    # Step 2: Delete Gateway and Targets
    print("\n[2/9] Deleting Gateway and Targets...")
    try:
        delete_gateway_and_targets(gateway_id)
        time.sleep(5)
    except Exception as e:
        print(f"Error in gateway deletion: {e}")
    
    # Step 3: Delete CloudWatch Log Deliveries
    print("\n[3/9] Deleting CloudWatch Log Deliveries...")
    try:
        delete_cloudwatch_log_deliveries(gateway_name)
        time.sleep(5)
    except Exception as e:
        print(f"Error deleting log deliveries: {e}")
    
    # Step 4: Delete CloudWatch Log Groups
    print("\n[4/9] Deleting CloudWatch Log Groups...")
    try:
        delete_cloudwatch_log_groups(gateway_id)
    except Exception as e:
        print(f"Error deleting log groups: {e}")
    
    # Step 5: Delete CloudTrail Trails
    print("\n[5/9] Deleting CloudTrail Trails...")
    try:
        delete_cloudtrail_trails()
        time.sleep(5)
    except Exception as e:
        print(f"Error deleting trails: {e}")
    
    # Step 6: Delete S3 Buckets
    print("\n[6/9] Deleting S3 Buckets...")
    try:
        buckets = s3_client.list_buckets()
        for bucket in buckets['Buckets']:
            bucket_name = bucket['Name']
            if 'agentcore-gateway' in bucket_name:
                empty_and_delete_s3_bucket(bucket_name)
    except Exception as e:
        print(f"Error deleting S3 buckets: {e}")
    
    # Step 7: Delete IAM Roles
    print("\n[7/9] Deleting IAM Roles...")
    try:
        delete_iam_roles()
    except Exception as e:
        print(f"Error deleting IAM roles: {e}")
    
    # Step 8: Delete Cognito Resources
    print("\n[8/9] Deleting Cognito Resources...")
    try:
        delete_cognito_resources(user_pool_id, client_id)
    except Exception as e:
        print(f"Error deleting Cognito resources: {e}")
    
    # Step 9: Delete Lambda Function
    print("\n[9/9] Deleting Lambda Function...")
    try:
        delete_lambda_function()
    except Exception as e:
        print(f"Error deleting Lambda function: {e}")
    
    print("\n" + "=" * 80)
    print("CLEANUP PROCESS COMPLETED")
    print("=" * 80)
    print("\nPlease verify in the AWS Console that all resources have been deleted.")
    print("Some resources may take a few minutes to fully delete.")
    print("\nVerification checklist:")
    print("  ✓ Amazon Bedrock AgentCore - Gateways")
    print("  ✓ Amazon Bedrock AgentCore - Runtime")
    print("  ✓ Amazon CloudWatch - Log Groups")
    print("  ✓ AWS CloudTrail - Trails")
    print("  ✓ Amazon S3 - Buckets")
    print("  ✓ AWS IAM - Roles")
    print("  ✓ Amazon Cognito - User Pools")
    print("  ✓ AWS Lambda - Functions")
    print("  ✓ Amazon ECR - Repositories")
